In [1]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F

from skorch import NeuralNetClassifier

### make dataset

In [2]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
print(X.shape,y.shape)
X = X.astype(np.float32)
y = y.astype(np.int64)

(1000, 20) (1000,)


### define network by pytorch

In [3]:
class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

### wrap your network and training

In [4]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

In [5]:
net.fit(X, y)
y_proba = net.predict_proba(X)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6746       0.6350        0.6699  0.0310
      2        0.6661       0.6650        0.6601  0.0285
      3        0.6413       0.6900        0.6629  0.0271
      4        0.6417       0.7050        0.6439  0.0271
      5        0.6327       0.7100        0.6352  0.0270
      6        0.6299       0.7000        0.6289  0.0268
      7        0.6137       0.6900        0.6292  0.0269
      8        0.6005       0.7000        0.6231  0.0269
      9        0.5962       0.7000        0.6127  0.0295
     10        0.5876       0.7200        0.6096  0.0279


### use pipeline

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

pipe.fit(X, y)
y_proba = pipe.predict_proba(X)

Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7186       0.5000        0.6940  0.0271
      2        0.6966       0.5100        0.6869  0.0273
      3        0.6867       0.5650        0.6825  0.0260
      4        0.6830       0.6600        0.6793  0.0268
      5        0.6780       0.6650        0.6763  0.0264
      6        0.6790       0.6250        0.6735  0.0265
      7        0.6711       0.6600        0.6694  0.0277
      8        0.6713       0.6700        0.6661  0.0303
      9        0.6690       0.6300        0.6623  0.0264
     10        0.6597       0.6550        0.6567  0.0258
